In [127]:
import pandas as pd
import numpy as np
from googletrans import Translator
from langdetect import detect
from unidecode import unidecode
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from gensim.models import Word2Vec 
import pickle

In [4]:
df=pd.read_csv(r'train.csv')
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [9]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
df['location'].value_counts().head(50)

USA                  104
New York              71
United States         50
London                45
Canada                29
Nigeria               28
UK                    27
Los Angeles, CA       26
India                 24
Mumbai                22
Washington, DC        21
Kenya                 20
Worldwide             19
Australia             18
Chicago, IL           18
California            17
Everywhere            15
New York, NY          15
California, USA       15
Florida               14
San Francisco         14
United Kingdom        14
Los Angeles           13
Indonesia             13
Washington, D.C.      13
Toronto               12
NYC                   12
Ireland               12
Seattle               11
Earth                 11
San Francisco, CA     11
Chicago               11
Texas                 10
London, England       10
Sacramento, CA        10
New York City         10
London, UK            10
Atlanta, GA           10
ss                    10
Dallas, TX             9


In [49]:
new_df=df[["text",'target']]

In [50]:
new_df

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


### Checking if other language is ther except engish

In [51]:
def lang(data):
    return detect(data)

new_df["Language"]=new_df["text"].apply(lang)

C:\Users\kannu\AppData\Local\Temp\ipykernel_16552\1091389273.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Language"]=new_df["text"].apply(lang)


In [52]:
new_df["Language"].value_counts()  #So tweets have been written in multiple languages.

en    7317
de      58
ca      25
da      22
af      20
it      19
fr      19
no      17
tl      15
nl      14
sv      12
et      11
cy       9
id       8
ro       6
es       6
vi       5
sl       5
so       5
pt       4
pl       4
sw       3
sq       2
hr       2
fi       2
lt       1
sk       1
tr       1
Name: Language, dtype: int64

### Converting other language in english

In [53]:
other_lang_index=new_df[new_df["Language"] !='en'].index
def trans(index):
    if index in other_lang_index:
        translator=Translator()
        return translator.translate(df['text'][index]).text
    else:
        return df['text'][index]

new_df['translated_lang']=new_df.reset_index()['index'].apply(trans)

C:\Users\kannu\AppData\Local\Temp\ipykernel_16552\421139417.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['translated_lang']=new_df.reset_index()['index'].apply(trans)


In [54]:
new_df.head()

,text,target,Language,translated_lang
0,Our Deeds are the Reason of this #earthquake M...,1,en,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada,1,en,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...,1,en,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or...",1,en,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...,1,en,Just got sent this photo from Ruby #Alaska as ...


In [58]:
def expand(data):
    return contractions.fix(data)

def accented(data):
    return unidecode(data)

def whitespace_remove(data):
    import re
    res=re.sub(r'\s+'," ",data)
    res=re.sub(r'\W+'," ",res)
    return res

df_white_space=new_df['translated_lang'].apply(whitespace_remove)
df_expand=df_white_space.apply(expand)
df_accented=df_expand.apply(accented)
df_accented.head()

0    Our Deeds are the Reason of this earthquake Ma...
1                Forest fire near La Ronge Sask Canada
2    All residents asked to shelter in place are be...
3    13 000 people receive wildfires evacuation ord...
4    Just got sent this photo from Ruby Alaska as s...
Name: translated_lang, dtype: object

In [59]:
stopwords_list=stopwords.words("english")


In [60]:
def cleaning(data):
    res=[word.lower() for word in word_tokenize(data) if (word.lower() not in stopwords_list) and (word not in punctuation) and (len(word)>2) and (word.isalpha()) ]
    return " ".join(res)

df_cleaned=df_accented.apply(cleaning)

### Performing lemmatization

In [64]:
def stem(data):
    text=[]
    for word in word_tokenize(data):
        lemma=WordNetLemmatizer()
        text.append(lemma.lemmatize(word,'v'))
    return " ".join(text)

df_lemmatized=df_cleaned.apply(stem)

### Word embedding

In [220]:
tfidf=TfidfVectorizer(max_df=.95,max_features=1000)
df_embedded=tfidf.fit_transform(df_lemmatized).A
df_final_embedded=pd.DataFrame(df_embedded,columns=tfidf.get_feature_names_out())

In [221]:
df_final_embedded.head()

,aba,abandon,abc,ablaze,accident,account,across,act,action,actually,...,yeah,year,years,yes,yet,york,young,youth,youtube,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [222]:
x=df_final_embedded
y=df['target']

In [223]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=5,stratify=y)


In [224]:
nb=MultinomialNB()
nb.fit(x_train,y_train)

MultinomialNB()

In [116]:
train=nb.score(x_train,y_train)
test=nb.score(x_test,y_test)
print(f"Training accuracy = {train}\nTesting accuracy = {test}")

Training accuracy = 0.8185550082101807
Testing accuracy = 0.7826657912015759


In [117]:
sv=SVC()
sv.fit(x_train,y_train)
train=sv.score(x_train,y_train)
test=sv.score(x_test,y_test)
print(f"Training accuracy = {train}\nTesting accuracy = {test}")


Training accuracy = 0.9229885057471264
Testing accuracy = 0.7971109652002626


In [91]:
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
train=rf.score(x_train,y_train)
test=rf.score(x_test,y_test)
print(f"Training accuracy = {train}\nTesting accuracy = {test}")

Training accuracy = 0.9796387520525451
Testing accuracy = 0.7852921864740644


In [92]:
xg=XGBClassifier()
xg.fit(x_train,y_train)
train=xg.score(x_train,y_train)
test=xg.score(x_test,y_test)
print(f"Training accuracy = {train}\nTesting accuracy = {test}")


Training accuracy = 0.861904761904762
Testing accuracy = 0.7984241628365069


### hypertuning RandomForest for bias-variance trade-off

In [112]:
hyp_parm={"n_estimators":np.arange(115,160),
          "criterion":["entropy",'gini'],
          "min_samples_split":np.arange(2,8),
          "min_samples_leaf":np.arange(2,8),
          "max_depth":np.arange(5,15),
          "random_state":[5]            
}

model=RandomForestClassifier()

random_cv=RandomizedSearchCV(model,hyp_parm)
random_cv.fit(x_train,y_train)


RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'min_samples_leaf': array([2, 3, 4, 5, 6, 7]),
                                        'min_samples_split': array([2, 3, 4, 5, 6, 7]),
                                        'n_estimators': array([115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
       154, 155, 156, 157, 158, 159]),
                                        'random_state': [5]})

In [110]:
random_cv.best_params_

{'random_state': 5,
 'n_estimators': 139,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_depth': 11,
 'criterion': 'gini'}

In [113]:
rf_tuned=random_cv.best_estimator_
rf_tuned.fit(x_train,y_train)
train=rf_tuned.score(x_train,y_train)
test=rf_tuned.score(x_test,y_test)
print(f"Training accuracy = {train}\nTesting accuracy = {test}")



Training accuracy = 0.7288998357963875
Testing accuracy = 0.7183191070256073


### Hypertuning svm

In [119]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
model=SVC()
grid=GridSearchCV(model,param_grid)
grid.fit(x_train,y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']})

In [120]:
grid.best_params_

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}

In [125]:
svc_tuned=grid.best_estimator_
svc_tuned.fit(x_train,y_train)
train=svc_tuned.score(x_train,y_train)
test=svc_tuned.score(x_test,y_test)
print(f"Training accuracy = {train}\nTesting accuracy = {test}")

Training accuracy = 0.9223316912972085
Testing accuracy = 0.7977675640183848


In [124]:
with open("svm_tuned.pkl","wb") as file:
    pickle.dump(svc_tuned,file)

In [128]:
lr=LogisticRegression()
lr.fit(x_train,y_train)
train=lr.score(x_train,y_train)
test=lr.score(x_test,y_test)
print(f"Training accuracy = {train}\nTesting accuracy = {test}")

Training accuracy = 0.8367816091954023
Testing accuracy = 0.7892317793827971


### Word embedding using word2vec

In [133]:
wor2vec_data=df_lemmatized.apply(lambda x:x.split()).to_list()

In [136]:
model=Word2Vec(window=10,min_count=2)
model.build_vocab(wor2vec_data)
model.train(wor2vec_data,total_examples=model.corpus_count,epochs=2)

(114775, 141916)

In [138]:
model.wv["hate"]

array([-0.03221368,  0.06727979,  0.00452291,  0.00260414,  0.002854  ,
       -0.05952301, -0.00176497,  0.07128789, -0.05456257, -0.02768043,
       -0.01932689, -0.101224  , -0.00110163,  0.00983945,  0.02876307,
       -0.04029382,  0.01273398, -0.0434704 ,  0.02287857, -0.03359714,
        0.04306038,  0.02485081,  0.01872555,  0.0010546 , -0.02335841,
       -0.00221399, -0.03301662, -0.01456346, -0.01574797, -0.01869644,
        0.00841339, -0.00613697, -0.0377964 , -0.04217856, -0.01561912,
        0.01567838,  0.0395291 , -0.02387262, -0.0312498 , -0.03826899,
        0.00685633, -0.05746465, -0.0219842 ,  0.018546  ,  0.02580196,
        0.00154032, -0.04088001,  0.01614777,  0.01533127,  0.00747005,
        0.02766312, -0.03020626,  0.00746111, -0.0038613 , -0.01431201,
       -0.00021262,  0.04254432, -0.01154005, -0.0417926 , -0.01358273,
       -0.03136577, -0.0048727 , -0.00583845,  0.00713577, -0.0011702 ,
        0.03165406,  0.01766482,  0.01679557, -0.01645768,  0.06

### Making code to get vector of each tweet or document

In [160]:
def word2vec(docs,model):
    all_doc_vector=[]
    for doc in docs:
        single_doc_vector=[]
        for word in doc:
            if word in  model.wv:
                try:
                    single_doc_vector.append(model.wv[word])
                except:
                    continue
                    
        if single_doc_vector:
            doc_array=np.array(single_doc_vector)
            mean_doc=doc_array.mean(axis=0)
            all_doc_vector.append(mean_doc)
        else:
            zero_vector=np.zeros(model.vector_size)
            
            all_doc_vector.append(zero_vector)
            
    return all_doc_vector



In [161]:
resultant_vector=word2vec(wor2vec_data,model)

In [168]:
list1=[]


In [176]:
x=pd.DataFrame(resultant_vector)

In [178]:
y=df['target']

In [190]:
x_train_wv,x_test_wv,y_train_wv,y_test_wv=train_test_split(x,y,test_size=.2,random_state=2,stratify=y)
rf_wv=RandomForestClassifier()
rf_wv.fit(x_train_wv,y_train_wv)
train_wv=rf_wv.score(x_train_wv,y_train_wv)
test_wv=rf_wv.score(x_test_wv,y_test_wv)
print(f"Training accuracy = {train_wv}\nTesting accuracy = {test_wv}")


Training accuracy = 0.9844006568144499
Testing accuracy = 0.6828627708470125


In [191]:
sv_wv=SVC()
sv_wv.fit(x_train_wv,y_train_wv)
train_wv=sv_wv.score(x_train_wv,y_train_wv)
test_wv=sv_wv.score(x_test_wv,y_test_wv)
print(f"Training accuracy = {train_wv}\nTesting accuracy = {test_wv}")

Training accuracy = 0.6024630541871921
Testing accuracy = 0.5981615233092581


In [186]:
hyp_parm_wv={"n_estimators":np.arange(115,160),
          "criterion":["entropy",'gini'],
          "min_samples_split":np.arange(2,8),
          "min_samples_leaf":np.arange(2,8),
          "max_depth":np.arange(5,15),
          "random_state":[5]            
}

model_wv=RandomForestClassifier()

random_cv=RandomizedSearchCV(model_wv,hyp_parm_wv)
random_cv.fit(x_train_wv,y_train_wv)

RandomizedSearchCV(estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'min_samples_leaf': array([2, 3, 4, 5, 6, 7]),
                                        'min_samples_split': array([2, 3, 4, 5, 6, 7]),
                                        'n_estimators': array([115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
       154, 155, 156, 157, 158, 159]),
                                        'random_state': [5]})

In [192]:
rf_tuned_wv=random_cv.best_estimator_
rf_tuned_wv.fit(x_train_wv,y_train_wv)
train_wv=rf_tuned_wv.score(x_train_wv,y_train_wv)
test_wv=rf_tuned_wv.score(x_test_wv,y_test_wv)
print(f"Training accuracy = {train_wv}\nTesting accuracy = {test_wv}")

Training accuracy = 0.9545155993431855
Testing accuracy = 0.6651346027577151


In [194]:
lr_wv=LogisticRegression()
lr_wv.fit(x_train_wv,y_train_wv)
train_wv=lr_wv.score(x_train_wv,y_train_wv)
test_wv=lr_wv.score(x_test_wv,y_test_wv)
print(f"Training accuracy = {train_wv}\nTesting accuracy = {test_wv}")

Training accuracy = 0.5756978653530378
Testing accuracy = 0.5751805646749836


### Hence best generalized model is naive bayes with tfidf word embedding having accuracies 


### Prediction of test data

In [210]:
df_test=pd.read_csv(r"test.csv")
test_text=pd.DataFrame(df_test['text'],columns=["text"])
test_text

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


### Will apply all preprocessing funcions on test data also.

In [211]:
test_text["Language"]=test_text["text"].apply(lang)

In [212]:
df_test['translated_lang']=df_test.reset_index()['index'].apply(trans)

In [213]:
df_white_space_test=df_test['translated_lang'].apply(whitespace_remove)
df_expand_test=df_white_space_test.apply(expand)
df_accented_test=df_expand_test.apply(accented)
df_accented_test.head()

0                   Just happened a terrible car crash
1    Heard about earthquake is different cities sta...
2    there is a forest fire at spot pond geese are ...
3                Apocalypse lighting Spokane wildfires
4        Typhoon Soudelor kills 28 in China and Taiwan
Name: translated_lang, dtype: object

In [214]:
df_cleaned_test=df_accented_test.apply(cleaning)

In [215]:
df_lemmatized_test=df_cleaned_test.apply(stem)

In [217]:
df_embedded_test=tfidf.transform(df_lemmatized_test).A
df_final_embedded_test=pd.DataFrame(df_embedded_test,columns=tfidf.get_feature_names_out())
df_final_embedded_test

,aba,abandon,abc,ablaze,accident,account,across,act,action,actually,...,yeah,year,years,yes,yet,york,young,youth,youtube,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.463903,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3259,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3260,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3261,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [225]:
result_test=nb.predict(df_final_embedded_test)
result_test

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [232]:
result=pd.DataFrame(result_test,columns=['target'])

In [233]:
result['id']=df_test['id']

In [235]:
result=pd.DataFrame(result,columns=['id','target'])

In [236]:
result

,id,target
0,0,1
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


In [237]:
result.to_csv("Result.csv",index=False)